In [1]:
import ast
import pandas as pd

In [12]:
import pandas as pd
import ast
all_results = {}
df = pd.read_csv('nlp(cultural).csv', sep=';',encoding ='utf-8')
df["correct_answers"] = df["correct_answers"].apply(ast.literal_eval)
#df["country"] = df["country"].apply(ast.literal_eval)
df["correct_answers_2"] = df["correct_answers_2"].apply(ast.literal_eval)
df

,id,question,type,subcategory,country,correct_answers,correct_answers_2,Unnamed: 7,Unnamed: 8
0,0,Which god was born from lotus flower that grew...,direct,religion,"[""India""]",[Brahma],[],NaN,NaN
1,1,"Who is depicted as a cosmic dancer, naked asce...",direct,religion,"[""India""]",[Shiva],[],NaN,NaN
2,2,"What is the name of the dish, which can be des...",direct,food,"[""India""]","[chicken tikka masala, chicken tikka, butter c...",[],NaN,NaN
3,3,"What is the name of the dish, popular in India...",direct,food,"[""India""]",[gobhi matar],[],NaN,NaN
4,4,What is the holiday that symbolises the spirit...,direct,culture,"[""India""]","[Diwali, Dipavali]",[],NaN,NaN
...,...,...,...,...,...,...,...,...,...
205,205,"In Russia, what is the traditional wooden nest...",direct,culture,['Russia'],"[Matryoshka, Matryoshka dolls]",[],NaN,NaN
206,206,"In Ukraine, what is the traditional embroidere...",direct,attire,['Ukraine'],[Vyshyvanka],[],NaN,NaN
207,207,"In Poland, what is the traditional filled dump...",direct,food,['Poland'],[Pierogi],[],NaN,NaN
208,208,"In Hungary, what is the paprika-spiced stew ma...",direct,food,['Hungary'],"[Goulash, Gulyás]",[],NaN,NaN


In [15]:
responses = []
for i, prompt in enumerate(df['question'][:5], start=1):
    response = chat(
        model='gemma3',
        messages=[{"role": "user", "content": prompt}]
    )
    responses.append(response.message.content)
    print(f"{i}/{len(df['question'][:5])} -> {response.message.content}")

1/5 -> That would be **Lakshmi**.

Here's the explanation of the fascinating story:

* **Vishnu's Cosmic Dream:** Vishnu was in deep meditation, resting on the serpent Shesha, who was coiled around a cosmic lotus.
* **The Dream:** Vishnu had a dream, and within that dream, a lotus flower began to bloom.
* **The Flower's Center:** From the center of that lotus flower sprang forth Lakshmi, the goddess of wealth, prosperity, and beauty.

This birth story is a central and beautifully symbolic part of Hindu mythology, representing the universe emerging from a divine source.

Do you want to know more about this story or perhaps other aspects of Hindu mythology?
2/5 -> The figure you’re describing is overwhelmingly associated with **Kundera**, a complex and fascinating deity within the Hindu tradition, particularly within Tantra and certain traditions of Shaivism. 

Here's a breakdown of why Kundera fits your description:

* **Cosmic Dancer:** Kundera is often depicted as a cosmic dancer, emb

In [9]:
import pandas as pd
import time
import os
import datetime
from ollama import chat

def generate_response_ollama(prompt, model, temperature=0.1):

    print(prompt)
    response = chat(
        model=model,
        messages=[{
            "role": "user",
            "content": prompt,
        }],
        options={
            "temperature": temperature
        }
    )

    return response.message.content or ""


def check_answer(response, expected_answers):
    if not response or not expected_answers:
        return False
 
    response_lower = response.lower()

    return any(
        expected.lower() in response_lower
        for expected in expected_answers
    )

def test(
    models_to_test,
    prompts,
    answers,
    answers2=None
):
    results = pd.DataFrame(index=range(len(prompts)), columns=models_to_test)
    result_prompts = pd.DataFrame(index=range(len(prompts)), columns=models_to_test)

    for model_name in models_to_test:
        print(f"\n{'='*60}")
        print(f"Testowanie modelu: {model_name}")
        print(f"{'='*60}")

        for i, prompt in enumerate(prompts, start=1):

            response = generate_response_ollama(
                prompt=prompt,
                model=model_name
            )
            result_prompts.loc[i-1, model_name] = response

            correct = check_answer(response, answers[i-1])
            if answers2 is not None and not correct and i < len(answers2):
                correct = check_answer(response, answers2[i])

            results.loc[i-1, model_name] = correct
            print(f"Wynik: {'✓' if correct else '✗'}")
        result_prompts.to_csv(
            f"{model_name[:5]}_prompts_results_{str(datetime.datetime.now())[:19]}.csv",
            encoding='utf-8-sig',
            index=False
)

        results.to_csv(
            f"{model_name[:5]}_results_{str(datetime.datetime.now())[:19]}.csv",
            encoding='utf-8-sig',
            index=False
)
    # Podsumowanie
    print(f"\n{'='*60}")
    print("PODSUMOWANIE")
    print(f"{'='*60}")

    for model_name in models_to_test:
        correct_count = results[model_name].sum()
        total = len(results)
        accuracy = (correct_count / total * 100) if total > 0 else 0
        print(f"{model_name}: {accuracy:.1f}% ({correct_count}/{total})")

    return results, result_prompts




In [14]:

models_to_test = [

"qwen3:0.6b",           
"mistral:latest",  
"llama3.1:latest",       
"qwen2.5:latest" ,        
"gemma3:1b",              
"gemma3:latest",          

    
    

]

results, result_prompts = test(models_to_test, df['question'], df['correct_answers'], df['correct_answers_2'])


Testowanie modelu: qwen3:0.6b
Which god was born from lotus flower that grew out of the navel of Vishnu
Wynik: ✗
Who is depicted as a cosmic dancer, naked ascetic, a yogi or half-male, half-female
Wynik: ✗
What is the name of the dish, which can be descripted as "Chicken marinated in a Yogurt tomato sauce"
Wynik: ✗
What is the name of the dish, popular in India, which can be described as "Cauliflower in a tomato sauce"
Wynik: ✗
What is the holiday that symbolises the spiritual victory of Dharma over Adharma
Wynik: ✗
What ethnic group is the largest in the Karala State
Wynik: ✗
What is the name of the god, prayed on a Indian-Bangladeshi border both by Muslims and Hindus
Wynik: ✗
What are names of 2 the most populous ethnic groups, living in a country where football national team is called "Red devils"
Wynik: ✗
In country, where in capital cities taxis are obligatory yellow, what is the name of the simple dish prepared with ants
Wynik: ✗
How is called the mean of transport which connect

OSError: [Errno 22] Invalid argument: 'qwen3_prompts_results_2026-01-20 10:58:59.csv'

In [156]:
message

ChatCompletionOutputMessage(role='assistant', content=None, reasoning=None, tool_call_id=None, tool_calls=[], reasoning_content="\nOkay, so the user is asking which god was born from a lotus flower that grew out of the navel of Vishnu. Let me start by recalling what I know about Hindu cosmology and the Trimurti. Vishnu is one of the principal deities, along with Brahma and Shiva. The story about Vishnu's navel and a lotus flower... I think this is related to the creation of the universe.\n\nWait, in some Hindu texts, it's said that during the creation, Vishnu was resting on the cosmic ocean. There's a story where a lotus grows from his navel, and from that lotus, different deities or divine beings are born. I remember the goddess Lakshmi is associated with this. But maybe there's another deity too?\n\nLet me think. In the Puranas, there's a story where Brahma is born from a lotus that emerged from Vishnu's navel. Wait, but Brahma")